In [ ]:
import ROOT as rt
import numpy as np
import plotly as pl
import plotly.graph_objects as go
from larcv import larcv
from larlite import larlite
import lardly
%load_ext autoreload 
%autoreload 2

In [ ]:
from lardly.detectoroutline import get_tpc_boundary_plot
from larlite import larutil

larutil.LArUtilConfig.SetDetector( larlite.geo.kSBND )

TPC_PLOT = {}
for ITPC in range(2):
    TPC_PLOT[ITPC] = get_tpc_boundary_plot(cryoid=0,tpcid=ITPC)
    

In [ ]:
infile = rt.TFile("testout_simchannelvoxelizer.root")
simchvox = infile.Get("simchannelvoxelizer")
nentries = simchvox.GetEntries()

In [ ]:
IENTRY = 0
simchvox.GetEntry(IENTRY)

In [ ]:
# PLOT TRUE POSITIONS

plot_list = []

for ITPC in range(2):
    coord  = simchvox.coordindex_v.at(ITPC).tonumpy()
    pos    = simchvox.truepos_v.at(ITPC).tonumpy()
    ancestor = simchvox.ancestor_v.at(ITPC).tonumpy()

    ancestor_list = np.unique(ancestor)
    
    for aid in ancestor_list:
        if aid==0:
            continue
            
        xpos = pos[ ancestor[:]==aid, : ]
        
        xcolor = np.zeros(3,dtype=np.dtype)
        xcolor[0] = np.random.randint(0,255)
        xcolor[1] = np.random.randint(0,255)
        xcolor[2] = np.random.randint(0,255)
        zcolor = "rgb(%d,%d,%d)"%(xcolor[0],xcolor[1],xcolor[2])

        simchvoxels = {
            "type":"scatter3d",
            "x": xpos[:,0],
            "y": xpos[:,1],
            "z": xpos[:,2],
            "mode":"markers",
            "name":"AID[%d]"%(aid),
            "marker":{"color":zcolor,"size":1,"opacity":0.8,"colorscale":'Viridis'},
        }

        plot_list.append( simchvoxels )
        plot_list.append( TPC_PLOT[ITPC] )

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

layout = go.Layout(
    title='simch',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 1},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

In [ ]:
ITPC = 0
charge = simchvox.charge_v.at(ITPC)
coord  = simchvox.coordindex_v.at(ITPC)
pdg    = simchvox.pdg_v.at(ITPC)
print(charge)

np_charge = charge.tonumpy()
np_coord  = coord.tonumpy()
np_pdg = pdg.tonumpy()
        
print("min charge: ",np.min(np_charge))
print("max charge: ",np.max(np_charge))
print("PDGs: ",np.unique(np_pdg))
        
simchvoxels = {
    "type":"scatter3d",
    "x": np_coord[:,0],
    "y": np_coord[:,1]*0.3,
    "z": np_coord[:,2]*0.3,
    "mode":"markers",
    "name":"simch",
    "marker":{"color":np_charge,"size":1,"opacity":0.8,"colorscale":'Viridis'},
}

plot_list = [ simchvoxels ]

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

layout = go.Layout(
    title='simch',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

In [ ]:
# PLOT BY PDG

ITPC = 0
coord  = simchvox.coordindex_v.at(ITPC)
pdg    = simchvox.pdg_v.at(ITPC)
print("PDG: ",pdg.shape)

np_coord  = coord.tonumpy()
np_pdg = pdg.tonumpy()
        
pdg_v = np.unique(np_pdg)
print("PDGs: ",pdg_v)

plot_list = []

for ipdg in pdg_v:
    
    xpos = np_coord[np_pdg[:]==ipdg,:]*0.3
    print("PDG=",pdg,": ",xpos.shape)
    print("xcoord min: ",np.min(xpos))
    print("xcoord max: ",np.max(xpos))
    
    xcolor = np.zeros(3,dtype=np.dtype)
    xcolor[0] = np.random.randint(0,255)
    xcolor[1] = np.random.randint(0,255)
    xcolor[2] = np.random.randint(0,255)
    zcolor = "rgb(%d,%d,%d)"%(xcolor[0],xcolor[1],xcolor[2])
        
    simchvoxels = {
        "type":"scatter3d",
        "x": xpos[:,0],
        "y": xpos[:,1],
        "z": xpos[:,2],
        "mode":"markers",
        "name":"PDG[%d]"%(ipdg),
        "marker":{"color":zcolor,"size":1,"opacity":0.8},
    }

    plot_list.append(simchvoxels)

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

layout = go.Layout(
    title='simch',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()

In [ ]:
# ZERO ANCESTOR VOXELS

ITPC = 0
charge = simchvox.charge_v.at(ITPC).tonumpy()
coord  = simchvox.coordindex_v.at(ITPC).tonumpy()
ancestor = simchvox.ancestor_v.at(ITPC).tonumpy()

xcoord = coord[ ancestor[:]==0, :]*0.3

ycoord = coord[ ancestor[:]!=0, :]*0.3

print("xcoord: ",xcoord.shape)
print("ycharge: ",xcoord.shape)
        
plot_noancestor = {
    "type":"scatter3d",
    "x": xcoord[:,0],
    "y": xcoord[:,1],
    "z": xcoord[:,2],
    "mode":"markers",
    "name":"simch",
    "marker":{"color":"rgb(255,0,0)","size":1,"opacity":0.8,"colorscale":'Viridis'},
}

plot_withancestor = {
    "type":"scatter3d",
    "x": ycoord[:,0],
    "y": ycoord[:,1],
    "z": ycoord[:,2],
    "mode":"markers",
    "name":"simch",
    "marker":{"color":"rgb(0,0,0)","size":1,"opacity":0.2,"colorscale":'Viridis'},
}

plot_list = [ plot_withancestor, plot_noancestor ]

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

layout = go.Layout(
    title='simch',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()


In [ ]:
# PLOT BY ANCESTOR

plot_list = []
skip_zero = True

for ITPC in range(2):
    coord    = simchvox.coordindex_v.at(ITPC).tonumpy()
    ancestor = simchvox.ancestor_v.at(ITPC).tonumpy()
    print(coord.shape)
    print(ancestor.shape)

    ancestor_v = np.unique(ancestor)
    print("num ancestor IDs: ",ancestor_v.shape)

    for aid in ancestor_v:
    
        if aid==0 and skip_zero:
            continue
    
        xpos = coord[ancestor[:]==aid,:]
        print("AncestorID=",aid,": ",xpos.shape)
    
        xcolor = np.zeros(3,dtype=np.dtype)
        xcolor[0] = np.random.randint(0,255)
        xcolor[1] = np.random.randint(0,255)
        xcolor[2] = np.random.randint(0,255)
        zcolor = "rgb(%d,%d,%d)"%(xcolor[0],xcolor[1],xcolor[2])
        
        if ITPC==0:
            xpos[:,0] = xpos[:,0]*(-1.0)
        
        simchvoxels = {
            "type":"scatter3d",
            "x": xpos[:,0],
            "y": xpos[:,1],
            "z": xpos[:,2],
            "mode":"markers",
            "name":"AID[%d]"%(aid),
            "marker":{"color":zcolor,"size":1,"opacity":0.8},
        }

        plot_list.append(simchvoxels)

axis_template = {
    "showbackground": True,
    "backgroundcolor": "rgba(10,10,10,0.1)",
    "gridcolor": "rgb(10, 10, 10,0.2)",
    "zerolinecolor": "rgb(10,10,10,0.4)",
}

layout = go.Layout(
    title='Ancestor ID',
    autosize=True,
    hovermode='closest',
    showlegend=False,
    scene= {
        "xaxis": axis_template,
        "yaxis": axis_template,
        "zaxis": axis_template,
        "aspectratio": {"x": 1, "y": 1, "z": 3},
        "camera": {"eye": {"x": -2, "y": 0.25, "z": 0.0},
                   "center":dict(x=0, y=0, z=0),
                   "up":dict(x=0, y=1, z=0)},
        "annotations": [],
    }
)

fig = go.Figure(data=plot_list, layout=layout)
fig.show()